In [1]:
import numpy as np
import pandas as pd

from copy import deepcopy
from functools import partial
import matplotlib.pyplot as plt
import optuna
import pickle
from sklearn.metrics import mean_squared_error
from tqdm import tqdm

In [2]:
submission_s1 = pd.read_csv('../output/submission_5_95_hokan_cleaning_100000tree_seed1.csv')
submission_s2 = pd.read_csv('../output/submission_5_95_hokan_cleaning_100000tree_seed2.csv')
submission_s3 = pd.read_csv('../output/submission_5_95_hokan_cleaning_100000tree_seed3.csv')
submission1 = (submission_s1['meter_reading'] + submission_s2['meter_reading'] + submission_s3['meter_reading']) /3
del submission_s1, submission_s2, submission_s3

In [3]:
submission_075_s1 = pd.read_csv('../output/submission_5_95_hokan_cleaning_100000tree_seed1_mul075.csv')
submission_075_s2 = pd.read_csv('../output/submission_5_95_hokan_cleaning_100000tree_seed2_mul075.csv')
submission_075_s3 = pd.read_csv('../output/submission_5_95_hokan_cleaning_100000tree_seed3_mul075.csv')
submission2 = (submission_075_s1['meter_reading'] + submission_075_s2['meter_reading'] + submission_075_s3['meter_reading']) /3
del submission_075_s1, submission_075_s2, submission_075_s3

In [4]:
submission_TE_s1 = pd.read_csv('../output/submission_TE_50000tree_seed1_mul075.csv')
submission_TE_s2 = pd.read_csv('../output/submission_TE_50000tree_seed2_mul075.csv')
submission_TE_s3 = pd.read_csv('../output/submission_TE_50000tree_seed3_mul075.csv')
submission3 = (submission_TE_s1['meter_reading'] + submission_TE_s2['meter_reading'] + submission_TE_s3['meter_reading']) / 3
del submission_TE_s1, submission_TE_s2, submission_TE_s3

In [5]:
submission_TE2_s1 = pd.read_csv('../output/submission_TE2_50000tree_seed1_mul075.csv')
submission_TE2_s2 = pd.read_csv('../output/submission_TE2_50000tree_seed2_mul075.csv')
submission_TE2_s3 = pd.read_csv('../output/submission_TE2_50000tree_seed3_mul075.csv')
submission4 = (submission_TE2_s1['meter_reading'] + submission_TE2_s2['meter_reading'] + submission_TE2_s3['meter_reading']) / 3
del submission_TE2_s1, submission_TE2_s2, submission_TE2_s3

In [6]:
submission_02_s1 = pd.read_csv('../output/submission_5_95_hokan_cleaning_100000tree_seed1_mul02.csv')
submission5 = submission_02_s1['meter_reading']
del submission_02_s1

In [7]:
submission6 = pd.read_csv('../output/submission_hokan_cleaning_100000tree_mul2.csv')
submission7  = pd.read_csv('../output/submission_leave7_300000tree.csv')
submission8  = pd.read_csv('../output/submission_5_95_hokan_cleaning_100000tree.csv')
submission9 = pd.read_csv('../output/submission_for_run.csv')
submission10 = pd.read_csv('../output/submission_5_95_hokan_cleaning_400000tree_lr001_mul075.csv')

In [8]:
submission11 = pd.read_csv('../output/fe2_lgbm.csv')
submission12 = pd.read_csv('../output/submission_tomioka.csv')
submission13 = pd.read_csv('../output/submission_half_and_half.csv')
submission14 = pd.read_csv('../output/submission_distill.csv')

In [9]:
# leaked_df = pd.read_csv('../input/leaked_data_all.csv', parse_dates=['timestamp'])
with open('../input/leak_data_drop_bad_rows2.pkl', 'rb') as f:
    leaked_df = pickle.load(f).rename(columns={'meter_reading': 'leaked_meter_reading'})

# leaked_df = pd.read_feather('../input/leak_data.feather').rename(columns={'meter_reading': 'leaked_meter_reading'})
leaked_df = leaked_df[['building_id','meter','timestamp', 'leaked_meter_reading']]
leaked_df = leaked_df.query('timestamp>=20170101')

with open('../input/building_metadata_raw.pkl', 'rb') as f:
    building_meta = pickle.load(f)

leaked_df = leaked_df.merge(building_meta[['building_id', 'site_id']], on='building_id', how='left')

leaked_df = leaked_df.query('~(meter==0 & site_id==0)')
# leaked_df = leaked_df.query('site_id==[2,4,15]')
# leaked_df = leaked_df.query('105<=building_id<=564 | 656<=building_id')

with open('../input/test_raw.pkl', 'rb') as f:
    test = pickle.load(f)

In [10]:
test['pred1'] = submission1.values
test['pred2'] = submission2.values
test['pred3'] = submission3.values
test['pred4'] = submission4.values
test['pred5'] = submission5.values
test['pred6'] = submission6['meter_reading']
test['pred7'] = submission7['meter_reading']
test['pred8'] = submission8['meter_reading']
test['pred9'] = submission9['meter_reading']
test['pred10'] = submission10['meter_reading']
test['pred11'] = submission11['meter_reading']
test['pred12'] = submission12['meter_reading']
test['pred13'] = submission13['meter_reading']
test['pred14'] = submission14['meter_reading']

del submission6, submission7, submission8, submission9, submission10, submission11, submission12, submission13, submission14

In [11]:
# test['pred11'] = np.exp(1) - 1

In [12]:
# pred_sorted1 = pd.DataFrame(np.sort(test.loc[:,'pred1':'pred13'].values, axis=1), columns=[f'pred{i}' for i in range(14, 27)])
# pred_sorted2 = pd.DataFrame(np.sort(test.loc[:,'pred1':'pred10'].values, axis=1), columns=[f'pred{i}' for i in range(27, 37)])

# test = pd.concat([test, pred_sorted1, pred_sorted2], axis=1)

# del pred_sorted1, pred_sorted2

In [13]:
test = test.merge(leaked_df, on=['building_id', 'meter', 'timestamp'], how='left')
N = test.columns.str.startswith('pred').sum()

In [14]:
# with open('../input/building_metadata_raw.pkl', 'rb') as f:
#     building_meta = pickle.load(f)


# test = test.merge(building_meta[['building_id', 'site_id']], on='building_id', how='left')

In [15]:
test_sub = test.copy()
test = test[~test['leaked_meter_reading'].isnull()]
test2017 = test.query('timestamp<20180101')
test2018 = test.query('20180101<=timestamp')

In [16]:
for i in range(1,N+1):
    print(i, np.sqrt(mean_squared_error(np.log1p(test2018['leaked_meter_reading']), np.log1p(test2018[f'pred{i}']))))

print()

for i in range(1,N+1):
    print(i, np.sqrt(mean_squared_error(np.log1p(test2017['leaked_meter_reading']), np.log1p(test2017[f'pred{i}']))))

print()
    
for i in range(1,N+1):
    print(i, np.sqrt(mean_squared_error(np.log1p(test['leaked_meter_reading']), np.log1p(test[f'pred{i}']))))


1 1.05707431984265
2 1.0558796429275272
3 1.0510705844554578
4 1.0527115582309234
5 1.085017795223617
6 1.062084119954559
7 1.089608889667971
8 1.060677174029206
9 1.081500810935177
10 1.0631875913803086
11 1.0670692750633455
12 1.074835895542593
13 1.0831985090552576
14 1.0827872912557106

1 0.9105108359275668
2 0.9103696794240971
3 0.9012662488544053
4 0.9044416426785011
5 0.958620671178696
6 0.9083304918286321
7 0.9520295444421947
8 0.9081599079972706
9 0.923402758171367
10 0.9309174130161622
11 0.9375401388043237
12 0.9523766285512328
13 0.9460503393895124
14 0.9524520973724483

1 0.9863821696123254
2 0.9856780561923182
3 0.9788988915843845
4 0.981243193567017
5 1.0236543886863338
6 0.9880595373342751
7 1.0230066442069852
8 0.9872263683022606
9 1.0054170589340188
10 0.9991207012008274
11 1.0042746771869266
12 1.0153402306340538
13 1.0168117958050467
14 1.0195831959447215


In [20]:
sid = 1
for i in range(1,N+1):
    print(i, np.sqrt(mean_squared_error(np.log1p(test2018.query(f'meter==0 & site_id=={sid}')['leaked_meter_reading']), 
                                        np.log1p(test2018.query(f'meter==0 & site_id=={sid}')[f'pred{i}']))))

print()

for i in range(1,N+1):
    print(i, np.sqrt(mean_squared_error(np.log1p(test2017.query(f'meter==0 & site_id=={sid}')['leaked_meter_reading']), 
                                        np.log1p(test2017.query(f'meter==0 & site_id=={sid}')[f'pred{i}']))))
       

1 1.0757427876594083
2 1.0676847649662704
3 1.0771935476110943
4 1.076498642841301
5 1.0347828208205818
6 1.0949802215560984
7 1.0761297040460147
8 1.0916569067281785
9 1.0984362592915258
10 1.0607135807932335
11 3.651170459130679

1 0.7941939381872285
2 0.7834770191804799
3 0.7928290071315696
4 0.7947646177287292
5 0.7421142944911797
6 0.8184734097791797
7 0.7939469230402673
8 0.813513235264502
9 0.8211185175276395
10 0.7754211863433329
11 3.684592577172206


In [17]:
focus_df = test2018

In [18]:
def preproceeding(submission, N):
    submission.loc[:,'pred1':'leaked_meter_reading'] = np.log1p(submission.loc[:,'pred1':'leaked_meter_reading'])
    g = submission.groupby('meter')
    sub_sub = [dict(), dict(), dict(), dict()]
    leak_sub = [dict(), dict(), dict(), dict()]
    leak_leak = [0,0,0,0]
    
    for meter in [3,2,1,0]:
        for i in tqdm(range(1,N+1)):
            leak_sub[meter][i] = sum(-2 * g.get_group(meter)['leaked_meter_reading'] * g.get_group(meter)[f'pred{i}'])
            for j in range(1,N+1):
                if i > j: 
                    sub_sub[meter][(i,j)] = sub_sub[meter][(j,i)]
                else:
                    sub_sub[meter][(i,j)] = sum(g.get_group(meter)[f'pred{i}'] * g.get_group(meter)[f'pred{j}'])
        
        leak_leak[meter] = (sum(g.get_group(meter)['leaked_meter_reading'] ** 2))
    
    return sub_sub, leak_sub, leak_leak

In [19]:
sub_sub, leak_sub, leak_leak = preproceeding(focus_df.copy(), N)

100%|██████████| 14/14 [01:11<00:00,  5.11s/it]


In [20]:
with open('../output/sub_sub1219_reproduce.pkl', 'wb') as f:
    pickle.dump(sub_sub, f)

In [21]:
with open('../output/leak_sub1219_reproduce.pkl', 'wb') as f:
    pickle.dump(leak_sub, f)

In [22]:
with open('../output/leak_leak1219_reproduce.pkl', 'wb') as f:
    pickle.dump(leak_leak, f)

In [23]:
count_itr = 0
def optimization(meter, sub_sub, leak_sub, leak_leak, length, W):
    
#     global count_itr
#     if count_itr%1000 == 0: print(count_itr, end=' ')
#     count_itr += 1
    
    loss_total = 0

    for i, a in enumerate(W, 1):
        for j, b in enumerate(W, 1):
            loss_total += a * b * sub_sub[meter][(i, j)]

    for i, a in enumerate(W, 1):
        loss_total += leak_sub[meter][i] * a

    loss_total += leak_leak[meter]
    
    return np.sqrt(loss_total / length)

In [24]:
np.random.seed(1)

score = [list(), list(), list(), list()]
weight = [list(), list(), list(), list()]

for meter in [0,1,2,3]:
    f = partial(optimization, meter, sub_sub, leak_sub, leak_leak, len(focus_df.query(f'meter=={meter}')))
    for i in tqdm(range(100000)):
        W = np.random.rand(N)

        to_zero = np.arange(N)
        np.random.shuffle(to_zero)

        W[to_zero[:np.random.randint(10)]] = 0

        W /= W.sum()
        W *= np.random.rand() * 0.3 + 0.8
        score[meter].append(f(W))
        weight[meter].append(W)
    score[meter] = np.array(score[meter])
    weight[meter] = np.array(weight[meter])

100%|██████████| 100000/100000 [00:15<00:00, 6606.05it/s]


In [29]:
# 今までのbest_score
# 0
# 0.6457341965029569 # 2,4,15
# 0.7670864405850898 # 1,2,15

# 1
# 1.1028821246797627

# 2
# 0.9266132278805228

# 3
# 1.1595900581550331


# new leak set
# 0.7229769339789924
# 1.4360871220006104
# 0.9269688704973097
# 1.1567657302973289

# 0.723733378801817
# 1.4325472242980108
# 0.9270139246582472
# 1.1560000215450374

# 0.7232916263657445
# 1.4257850382841468
# 0.926444086189883
# 1.1553662804048155

In [25]:
for meter in [0,1,2,3]:
    print(score[meter].min())
    print(weight[meter][score[meter].argmin()].sum())
    print()

0.7236473125816902
0.9828885886233922

1.4309527962488617
0.9567377754250824

0.9266339383485149
1.0064676416257325

1.1566428189821238
0.9302832330250349



In [30]:
for meter in [0,1,2,3]:
    print(score[meter].min())
    print(weight[meter][score[meter].argmin()].sum())
    print()

0.7249065578372264
0.9798872704193033

1.4397967438975836
1.0984834964889494

0.9331648160641498
1.0910739378990326

1.1717147955151652
0.9335901334642571



In [26]:
for meter in [0,1,2,3]:
#     for i in range(N):
    print(weight[meter][score[meter].argmin()])
    print()

[0.         0.10092835 0.20018035 0.         0.18654509 0.
 0.         0.00840609 0.21663667 0.         0.         0.12658947
 0.14360258 0.        ]

[0.         0.09565136 0.04687625 0.06225666 0.         0.
 0.         0.         0.         0.16233157 0.19879467 0.18168248
 0.         0.20914479]

[0.         0.         0.21024257 0.17424361 0.         0.05958822
 0.         0.         0.05475527 0.03109518 0.2207995  0.18694858
 0.06879473 0.        ]

[0.23867828 0.1178646  0.10729524 0.         0.         0.
 0.1001939  0.         0.         0.         0.13644665 0.
 0.22980456 0.        ]



In [32]:
for meter in [0,1,2,3]:
#     for i in range(N):
    print(weight[meter][score[meter].argmin()])
    print()

[0.         0.         0.34936335 0.         0.19842256 0.
 0.         0.         0.40810671 0.02399465 0.        ]

[0.         0.         0.41478892 0.34445995 0.         0.
 0.         0.         0.04120542 0.1177256  0.1803036 ]

[0.         0.         0.27079235 0.38448444 0.         0.09497096
 0.         0.         0.23483615 0.         0.10599004]

[0.         0.63973471 0.         0.         0.         0.
 0.22908722 0.         0.         0.06476821 0.        ]



In [33]:
def new_pred(test_sub, weight, N):
    pred_new = list()
    for meter in [0,1,2,3]:
        test_sub_m = test_sub.query(f'meter=={meter}')
        ensemble_m = sum([np.log1p(test_sub_m[f'pred{i+1}']) * weight[meter][score[meter].argmin()][i] for i in range(N)])
        pred_new.append(ensemble_m)

    pred_new = pd.concat(pred_new).sort_index()
    return np.expm1(pred_new)

In [34]:
ensembled_pred1 = new_pred(test_sub, weight, N)

In [35]:
print(np.sqrt(mean_squared_error(np.log1p(test2018['leaked_meter_reading']), np.log1p(ensembled_pred1.loc[test2018.index]))))
print(np.sqrt(mean_squared_error(np.log1p(test2017['leaked_meter_reading']), np.log1p(ensembled_pred1.loc[test2017.index]))))
print(np.sqrt(mean_squared_error(np.log1p(test['leaked_meter_reading']), np.log1p(ensembled_pred1.loc[test.index]))))

1.0294448479864227
0.890726560019274
0.9624590705593496


In [30]:
print(np.sqrt(mean_squared_error(np.log1p(test2018['leaked_meter_reading']), np.log1p(ensembled_pred1.loc[test2018.index]))))
print(np.sqrt(mean_squared_error(np.log1p(test2017['leaked_meter_reading']), np.log1p(ensembled_pred1.loc[test2017.index]))))
print(np.sqrt(mean_squared_error(np.log1p(test['leaked_meter_reading']), np.log1p(ensembled_pred1.loc[test.index]))))

1.0212183713099725
0.8871785002628109
0.9564247823586295


In [31]:
print(np.sqrt(mean_squared_error(np.log1p(test2018['leaked_meter_reading']), np.log1p(ensembled_pred1.loc[test2018.index]))))
print(np.sqrt(mean_squared_error(np.log1p(test2017['leaked_meter_reading']), np.log1p(ensembled_pred1.loc[test2017.index]))))
print(np.sqrt(mean_squared_error(np.log1p(test['leaked_meter_reading']), np.log1p(ensembled_pred1.loc[test.index]))))

1.0253040456482883
0.887709111556231
0.9588498640851298


In [32]:
# 0.9455117442226527
# 0.7985375584144594

# 0.9404366275680945
# 0.8062714386695903

#0.9408301210841344
#0.8029139931610979
#all 0.8744504105324552

# 0.9382531143644084
# 0.8031481393084626
# 0.873175711300433

In [36]:
new_submission = pd.read_csv('../input/sample_submission.csv')

In [37]:
new_submission['meter_reading'] = ensembled_pred1.values
#                                       submission1['meter_reading'] * arr[arr[:,0].argmin()][1][0] \
#                                     + submission2['meter_reading'] * arr[arr[:,0].argmin()][1][1] \
#                                     + submission3['meter_reading'] * arr[arr[:,0].argmin()][1][2]

In [38]:
new_submission.to_csv('../output/submission_my_leak_validation.csv', index=False)

In [30]:
new_submission

,row_id,meter_reading
0,0,153.789032
1,1,73.405287
2,2,6.593431
3,3,246.250961
4,4,1169.665890
5,5,12.276495
6,6,111.057664
7,7,371.991712
8,8,782.021891
9,9,325.093032
